In [16]:
from pathlib import Path
import pandas as pd
import altair as alt

In [28]:
configs = {'LP-2fu-nofuse':'LP-NF-2', 'LP-2fu-fuse':'LP-F-2',
           'LP-1fu-nofuse':'LP-NF-1', 'LP-1fu-fuse':'LP-F-1',
           'HP-3fu-nofuse':'HP-NF-3', 'HP-3fu-fuse':'HP-F-3',
           'HP-2fu-nofuse':'HP-NF-2', 'HP-2fu-fuse':'HP-F-2',  
           'HP-1fu-nofuse':'HP-NF-1', 'HP-1fu-fuse':'HP-F-1'}
configs_order = ['LP-NF-2', 'LP-F-2', 'LP-NF-1', 'LP-F-1',
                 'HP-NF-3', 'HP-F-3', 'HP-NF-2', 'HP-F-2', 'HP-NF-1', 'HP-F-1']

In [33]:
benchmarks = {
    'hmmer-small_0':'hmmer',
    #'libquantum-small_0':'libquantum',
    #'blackscholes_simsmall_1_0':'blackscholes',
    #'freqmine_simsmall_1_0':'freqmine',
    #'swaptions_simsmall_1_0':'swaptions',
    'integerNN_0':'integerNN',
    'streamvbyte_0':'streamvbyte',
    'img_cartoon_image1_0':'cartoon',
    'img_cartoon_image2_0':'cartoon',
    'img_cartoon_image3_0':'cartoon',
    'img_cartoon_image4_0':'cartoon',
    'img_cartoon_image5_0':'cartoon',
    'img_canny_image1_0':'canny',
    'img_canny_image2_0':'canny',
    'img_canny_image3_0':'canny',
    'img_canny_image4_0':'canny',
    'img_canny_image5_0':'canny',
    #'img_hist_image1_0':'hist',
    #'img_hist_image2_0':'hist',
    #'img_hist_image3_0':'hist',
    #'img_hist_image4_0':'hist',
    #'img_hist_image5_0':'hist',
    'img_integral_image1_0':'img_integral',
    'img_integral_image2_0':'img_integral',
    'img_integral_image3_0':'img_integral',
    'img_integral_image4_0':'img_integral',
    'img_integral_image5_0':'img_integral',
    'conv_image1_0':'conv',
    'conv_image2_0':'conv',
    'conv_image3_0':'conv',
    'conv_image4_0':'conv',
    'conv_image5_0':'conv',
    'img_erode_image1_0':'erode',
    'img_erode_image2_0':'erode',
    'img_erode_image3_0':'erode',
    'img_erode_image4_0':'erode',
    'img_erode_image5_0':'erode',
    'img_median_image1_0':'median',
    'img_median_image2_0':'median',
    'img_median_image3_0':'median',
    'img_median_image4_0':'median',
    'img_median_image5_0':'median',
    #'img_yuv444_image1_0':'img_yuv444',
    #'img_yuv444_image2_0':'img_yuv444',
    #'img_yuv444_image3_0':'img_yuv444',
    #'img_yuv444_image4_0':'img_yuv444',
    #'img_yuv444_image5_0':'img_yuv444',
    'amax_cols_normal_16bit_0_0':'amax',
    'amax_cols_normal_16bit_1_0':'amax',
    'amax_cols_normal_16bit_2_0':'amax',
    'amax_cols_normal_16bit_3_0':'amax',
    'amax_cols_normal_16bit_4_0':'amax',
    #'asum_cols_normal_16bit_0_0':'asum',
    #'asum_cols_normal_16bit_1_0':'asum',
    #'asum_cols_normal_16bit_2_0':'asum',
    #'asum_cols_normal_16bit_3_0':'asum',
    #'asum_cols_normal_16bit_4_0':'asum',
    #'gemv_normal_16bit_0_0':'gemv',
    #'gemv_normal_16bit_1_0':'gemv',
    #'gemv_normal_16bit_2_0':'gemv',
    #'gemv_normal_16bit_3_0':'gemv',
    #'gemv_normal_16bit_4_0':'gemv',
    #'ger_normal_16bit_0_0':'ger',
    #'ger_normal_16bit_1_0':'ger',
    #'ger_normal_16bit_2_0':'ger',
    #'ger_normal_16bit_3_0':'ger',
    #'ger_normal_16bit_4_0':'ger',
    'sqnrm2_cols_normal_16bit_0_0':'sqnrm2',
    'sqnrm2_cols_normal_16bit_1_0':'sqnrm2',
    'sqnrm2_cols_normal_16bit_2_0':'sqnrm2',
    'sqnrm2_cols_normal_16bit_3_0':'sqnrm2',
    'sqnrm2_cols_normal_16bit_4_0':'sqnrm2',
    'gemm_normal_16bit_0_0':'gemm',
    'gemm_normal_16bit_1_0':'gemm',
    'gemm_normal_16bit_2_0':'gemm',
    'gemm_normal_16bit_3_0':'gemm',
    'gemm_normal_16bit_4_0':'gemm',
    'fft_normal_8bit_0_0':'fft',
    'fft_normal_8bit_1_0':'fft',
    'fft_normal_8bit_2_0':'fft',
    'fft_normal_8bit_3_0':'fft',
    'fft_normal_8bit_4_0':'fft',
}

bench_order = ['hmmer', 'streamvbyte', 'integerNN', 'cartoon', 'sqnrm2', 'amax', 'gemm', 'fft', 'conv', 'median', 'img_integral', 'canny', 'erode']
bench_row = {'hmmer':1, 'streamvbyte':1, 'integerNN':1, 'cartoon':1, 'sqnrm2':1, 'amax':1, 'gemm':1, 'fft':2, 'conv':2, 'median':2, 'img_integral':2, 'canny':2, 'erode':2}

In [19]:
stats = Path('./stats/')
stats = stats.glob('**/roi.txt')
stats = list(stats)
stats = [p.parents[0] for p in stats]
#stats

In [66]:
df = []

for s in stats:
    bench  = s.name
    config = s.parents[1].name
    
    if bench in benchmarks.keys() and config in configs.keys():
        try:
            usage = pd.read_csv(s/'simd_fu_used.csv')
        except EmptyDataError:
            continue

        usage = usage.drop(columns=['samples', 'mean', 'stdev', 'underflows', 'overflows', 'min_value', 'max_value'])
        usage = 100 * usage / int(usage['total'])
        usage = usage.drop(columns=['0', 'total'])
        usage['Benchmark'] = bench
        usage['Config'] = config
        usage = usage.melt(id_vars=['Benchmark', 'Config'], var_name='Var', value_name='Percentage')

        df.append(usage)
        
df = pd.concat(df)
df['Config'] = df['Config'].replace(configs)
df = df.set_index(['Config', 'Benchmark', 'Var'])
df = df.groupby(by=[None, benchmarks, None], level=['Config', 'Benchmark', 'Var']).mean()
df = df.reset_index()
df['Row'] = df['Benchmark'].replace(bench_row)
df

,Config,Benchmark,Var,Percentage,Row
0,HP-F-1,amax,1,40.233602,1
1,HP-F-1,canny,1,49.425179,2
2,HP-F-1,cartoon,1,50.466121,1
3,HP-F-1,conv,1,82.048139,2
4,HP-F-1,erode,1,92.826160,2
...,...,...,...,...,...
214,LP-NF-2,median,2,65.612871,2
215,LP-NF-2,sqnrm2,1,21.026728,1
216,LP-NF-2,sqnrm2,2,0.343763,1
217,LP-NF-2,streamvbyte,1,18.172308,1


In [99]:
usage_plot = alt.Chart(df).mark_bar(size=16).encode(
    y=alt.Y(
        'Percentage:Q',
        scale=alt.Scale(domain=(0, 100)),
        title="Percentage of execution cycles [%]"
    ),
    x=alt.X(
        'Config:N',
        sort=configs_order,
        title=""
    ),
    column=alt.Column(
        'Benchmark:N',
        sort=bench_order,
        spacing=8
    ),
    color=alt.Color(
        'Var:O',
        scale=alt.Scale(scheme='yellowgreenblue'),
        title='Number of active SIMD units'
    ),
).properties(
    height=alt.Step(19)
).facet(
    row=alt.Row(
        'Row:O',
        title=""
    )
).configure_axis(
    labelFontSize=17, titleFontSize=20
).configure_header(
    labelFontSize=21, titleFontSize=21
).configure_legend(
    titleFontSize=21, labelFontSize=21, orient='top', titleLimit=0
)
usage_plot

alt.FacetChart(...)

In [100]:
#usage_plot.save('plots/simd_usage.svg', webdriver='firefox')